<br>
    <center>МИНИСТЕРСТВО НАУКИ И ВЫСШЕГО ОБРАЗОВАНИЯ</center>
    <center>федеральное государственное автономное образовательное учреждение высшего образования </center> <center>«Самарский национальный исследовательский университет имени академика С.П. Королева»</center>
    <center>(Самарский университет)</center> </br>

<br/>

<br>
<center>Институт 	     информатики и кибернетики</center>                                                   	  
<center>Кафедра 	     технической кибернетики</center>                                                              	
</br>
<br/>

<br>

<br/>
<br/>
<br/>
<br>
<center>ОТЧЕТ</center>
<center>по лабораторной работе №3</center>

<center>«Введение в Spark с использованием Python»</center>
<br/>
<center>по дисциплине <strong>«Большие данные»</strong></center>
<br/>
<center></center>
</br>
<br/>
<br/>
<br/>



<br>
<p style="text-align:right;">Выполнил: Васильев А.С.
<br>6131-010402D
<br>    
<br>Преподаватель: Попов С.Б.
</p>
<br/>
<br/>
<br/>
    <br/>
<br/>
<br/>
<center>Самара 2025</center>


In [1]:
import os

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
os.environ["HADOOP_HOME"] = "C:/hadoop"
os.environ["hadoop.home.dir"] = "C:/hadoop"
os.environ["PATH"] = f"C:/hadoop/bin;{os.environ.get('PATH', '')}"

print("Переменные окружения установлены")

Переменные окружения установлены


In [2]:
import socketserver
print(socketserver.__file__)
print(dir(socketserver))

import pyspark
print(f"Версия PySpark: {pyspark.__version__}")

c:\Anaconda\envs\pyspark_env\lib\socketserver.py
['BaseRequestHandler', 'BaseServer', 'BufferedIOBase', 'DatagramRequestHandler', 'StreamRequestHandler', 'TCPServer', 'ThreadingMixIn', 'ThreadingTCPServer', 'ThreadingUDPServer', 'UDPServer', '_NoThreads', '_ServerSelector', '_SocketWriter', '_Threads', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '__version__', 'os', 'selectors', 'socket', 'sys', 'threading', 'time']
Версия PySpark: 3.5.0


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder \
    .appName("KafkaRead") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

print("Spark сессия создана успешно")

print(spark.version)

Spark сессия создана успешно
3.5.0


In [4]:
spark

In [5]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

In [6]:
kafka_servers = "localhost:9092"

In [7]:
topic1 = 'rides1'

In [8]:
kafka_servers = "localhost:9092"

df = spark \
    .read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_servers) \
    .option("subscribe", topic1) \
    .load()

df = df.withColumn('key_str', df['key'].cast('string').alias('key_str')).drop(
    'key').withColumn('value_str', df['value'].cast('string').alias('value_str')).drop('value')

df.show(5)

+------+---------+------+--------------------+-------------+-------+--------------------+
| topic|partition|offset|           timestamp|timestampType|key_str|           value_str|
+------+---------+------+--------------------+-------------+-------+--------------------+
|rides1|        0|     0|2025-12-19 01:32:...|            0|      1|1,2020-07-01 00:2...|
|rides1|        0|     1|2025-12-19 01:32:...|            0|      1|1,2020-07-01 00:0...|
|rides1|        0|     2|2025-12-19 01:32:...|            0|      2|2,2020-07-01 00:1...|
|rides1|        0|     3|2025-12-19 01:32:...|            0|      2|2,2020-07-01 00:3...|
|rides1|        0|     4|2025-12-19 01:32:...|            0|      2|2,2020-07-01 00:3...|
+------+---------+------+--------------------+-------------+-------+--------------------+
only showing top 5 rows



In [9]:
df.tail(5)

[Row(topic='rides1', partition=0, offset=261, timestamp=datetime.datetime(2025, 12, 19, 1, 32, 6, 66000), timestampType=0, key_str='1', value_str='1,2020-07-01 01:48:38,2020-07-01 01:51:55,1,1.30,1,N,140,75,2,5.5,3,0.5,0,0,0.3,9.3,2.5'),
 Row(topic='rides1', partition=0, offset=262, timestamp=datetime.datetime(2025, 12, 19, 1, 32, 6, 66000), timestampType=0, key_str='1', value_str='1,2020-07-01 01:55:54,2020-07-01 02:05:23,1,3.20,1,N,236,230,2,11.5,3,0.5,0,0,0.3,15.3,2.5'),
 Row(topic='rides1', partition=0, offset=263, timestamp=datetime.datetime(2025, 12, 19, 1, 32, 6, 66000), timestampType=0, key_str='1', value_str='1,2020-07-01 01:48:27,2020-07-01 01:52:21,1,.30,1,N,158,249,1,4.5,3,0.5,1.65,0,0.3,9.95,2.5'),
 Row(topic='rides1', partition=0, offset=264, timestamp=datetime.datetime(2025, 12, 19, 1, 32, 6, 66000), timestampType=0, key_str='1', value_str='1,2020-07-01 01:49:27,2020-07-01 01:55:52,1,.80,1,N,140,237,2,6.5,3,0.5,0,0,0.3,10.3,2.5'),
 Row(topic='rides1', partition=0, offset

In [10]:
from pyspark.sql.functions import split, col

parsed_df = df.withColumn(
    "parsed", split(col("value_str"), ",")
).select(
    col("parsed")[0].cast("int").alias("vendor_id"),
    col("parsed")[3].cast("int").alias("passenger_count"),
    col("parsed")[4].cast("float").alias("trip_distance"),
    col("parsed")[16].cast("float").alias("total_amount"))

parsed_df.show(5)

+---------+---------------+-------------+------------+
|vendor_id|passenger_count|trip_distance|total_amount|
+---------+---------------+-------------+------------+
|        1|              1|          1.5|         9.3|
|        1|              1|          9.5|        27.8|
|        2|              1|         5.85|        22.3|
|        2|              1|          1.9|       14.16|
|        2|              1|         1.25|         7.8|
+---------+---------------+-------------+------------+
only showing top 5 rows



In [11]:
from pyspark.sql.functions import sum as spark_sum, count

result_spark = parsed_df.groupBy("vendor_id").agg(
    count("*").alias("trip_count"),
    spark_sum("passenger_count").alias("total_passengers"),
    spark_sum("trip_distance").alias("total_distance"),
    spark_sum("total_amount").alias("total_amount_sum")
).orderBy("vendor_id")

print("=== Результаты через Spark DataFrame ===")
result_spark.show()

=== Результаты через Spark DataFrame ===
+---------+----------+----------------+------------------+------------------+
|vendor_id|trip_count|total_passengers|    total_distance|  total_amount_sum|
+---------+----------+----------------+------------------+------------------+
|        1|        87|             101|394.29999962449074|1976.6299829483032|
|        2|       179|             242| 744.6499975193292|3709.1699678897858|
+---------+----------+----------------+------------------+------------------+



In [12]:
parsed_df.createOrReplaceTempView("rides")

result_sql = spark.sql("""
    SELECT 
        vendor_id,
        COUNT(*) as trip_count,
        SUM(passenger_count) as total_passengers,
        SUM(trip_distance) as total_distance,
        SUM(total_amount) as total_amount_sum
    FROM rides
    GROUP BY vendor_id
    ORDER BY vendor_id
""")

print("=== Результаты через SQL ===")
result_sql.show()

=== Результаты через SQL ===
+---------+----------+----------------+------------------+------------------+
|vendor_id|trip_count|total_passengers|    total_distance|  total_amount_sum|
+---------+----------+----------------+------------------+------------------+
|        1|        87|             101|394.29999962449074|1976.6299829483032|
|        2|       179|             242| 744.6499975193292|3709.1699678897858|
+---------+----------+----------------+------------------+------------------+



In [13]:
import pyspark.pandas as ps

ps_df = parsed_df.pandas_api()

result_ps = ps_df.groupby("vendor_id").agg(
    trip_count=("vendor_id", "count"), 
    total_passengers=("passenger_count", "sum"),
    total_distance=("trip_distance", "sum"),
    total_amount_sum=("total_amount", "sum")
).reset_index()

result_ps = ps_df.groupby("vendor_id").agg(
    trip_count=("vendor_id", "count"),
    total_passengers=("passenger_count", "sum"),
    total_distance=("trip_distance", "sum"),
    total_amount_sum=("total_amount", "sum")
).reset_index()

print("=== Результаты через pandas-on-Spark ===")
print(result_ps)

result_ps2 = ps_df.groupby("vendor_id").agg({
    "passenger_count": "sum",
    "trip_distance": "sum",
    "total_amount": "sum"
})

trip_counts = ps_df.groupby("vendor_id").size().rename("trip_count")
result_ps2 = result_ps2.join(trip_counts).reset_index()

result_ps2.columns = ["vendor_id", "total_passengers", "total_distance", "total_amount_sum", "trip_count"]

result_ps2 = result_ps2[["vendor_id", "trip_count", "total_passengers", "total_distance", "total_amount_sum"]]

print("\n=== Результаты через pandas-on-Spark (способ 2) ===")
print(result_ps2)

=== Результаты через pandas-on-Spark ===
   vendor_id  trip_count  total_passengers  total_distance  total_amount_sum
0          1          87               101      394.300000       1976.629983
1          2         179               242      744.649998       3709.169968

=== Результаты через pandas-on-Spark (способ 2) ===
   vendor_id  trip_count  total_passengers  total_distance  total_amount_sum
0          1          87               101      394.300000       1976.629983
1          2         179               242      744.649998       3709.169968


In [14]:
pandas_df = parsed_df.toPandas()

result_pandas = pandas_df.groupby("vendor_id").agg({
    "passenger_count": "sum",
    "trip_distance": "sum",
    "total_amount": "sum"
}).reset_index()

trip_counts = pandas_df.groupby("vendor_id").size().reset_index()
trip_counts.columns = ["vendor_id", "trip_count"]  

result_pandas = result_pandas.merge(trip_counts, on="vendor_id")

result_pandas = result_pandas.rename(columns={
    "passenger_count": "total_passengers",
    "trip_distance": "total_distance",
    "total_amount": "total_amount_sum"
})

result_pandas = result_pandas[["vendor_id", "trip_count", "total_passengers", "total_distance", "total_amount_sum"]]

print("=== Результаты через pandas ===")
print(result_pandas)

=== Результаты через pandas ===
   vendor_id  trip_count  total_passengers  total_distance  total_amount_sum
0          1          87               101      394.299988       1976.630005
1          2         179               242      744.650024       3709.169922


In [15]:
pandas_df

,vendor_id,passenger_count,trip_distance,total_amount
0,1,1,1.50,9.300000
1,1,1,9.50,27.799999
2,2,1,5.85,22.299999
3,2,1,1.90,14.160000
4,2,1,1.25,7.800000
...,...,...,...,...
261,1,1,1.30,9.300000
262,1,1,3.20,15.300000
263,1,1,0.30,9.950000
264,1,1,0.80,10.300000


In [17]:
result_pd = result_spark.toPandas()
result_pd.to_csv("results_by_vendor.csv", index=False)
print("Результаты сохранены в C:/tmp/results_by_vendor.csv")
print(result_pd)

Результаты сохранены в C:/tmp/results_by_vendor.csv
   vendor_id  trip_count  total_passengers  total_distance  total_amount_sum
0          1          87               101      394.300000       1976.629983
1          2         179               242      744.649998       3709.169968


In [18]:
spark.stop()